# Try MWAS with covariates prepared in various ways

In this version, we only use the SNPs (from new files) also in old files

# Pick the regions we will test

In [1]:
library(data.table)

prev_hits <- fread("
Chr     pos        old_z       old_p
11   38247902        -27.1535308285104       2.30024742330298e-162
2    47933357        18.3327793004811        4.53147699327216e-75
7    1987910         10.1255085321387        4.25759742738181e-24
7    1987896         10.1255085321387        4.25759742738189e-24
7    1987797         10.0241523897721        1.19379483108027e-23
7    1987778         10.0105851568856        1.36940148731912e-23
12   2194742         -10.0072634920486       1.41615523554955e-23
")


In [2]:
#df <- fread("09.5-OUT_matched_SNP_meth_cov_chunked_EXPANSE_a2.csv")

In [3]:
df <- fread("09-OUT_matched_SNP_meth_cov_a2.csv")

## Try original code

In [4]:
###### model: learn elastic net model on training data 
######---------Input: trainX, trainY
######---------Return: selected features and coefficents

# original
elastic.net <- function(trainX,trainY){
    if(nrow(trainX)!=length(trainY)){
            stop("Number of observations is differerent")
    } 

    # optimize alpha---mixing parameter  
    a <- 0.5
    search <- foreach(ai = a, .combine = rbind) %dopar% {
        cv.fit <- cv.glmnet(
                        trainX,
                        trainY,
                        nfold = 5,
                        type.measure = "mse",
                        paralle = TRUE,
                        alpha = ai
                        )
        data.frame(
                        cvm = min(cv.fit$cvm),
                        lambda = cv.fit$lambda.min,
                        alpha = ai
                        )
        } 
    cv.opt <- search[search$cvm == min(search$cvm),] 

        # fit model by optimized alpha and lambda
        yfit = glmnet(
        trainX,
        trainY,
        lambda = cv.opt$lambda,
        alpha = cv.opt$alpha
                )       
        idf <- coef(yfit)
        idx <- which(idf != 0)
        selectf <- data.frame(
                features = idf@Dimnames[[1]][idx], 
                coefs = idf [idx]
        )
}

# modified to use lambda 1se and appropriate cvm
# elastic.net <- function(trainX,trainY){
#     if(nrow(trainX)!=length(trainY)){
#             stop("Number of observations is differerent")
#     } 

#     # optimize alpha---mixing parameter  
#     a <- 0.5
#     search <- foreach(ai = a, .combine = rbind) %dopar% {
#         set.seed(42)
#         cv.fit <- cv.glmnet(
#                         trainX,
#                         trainY,
#                         nfold = 5,
#                         type.measure = "mse",
#                         paralle = TRUE,
#                         alpha = ai
#                         )
#         print(paste0("Dim of trainX: ", dim(trainX)))
#         print(paste0("Len of trainY: ", length(trainY)))
#         coef_matrix <- as.matrix(coef(cv.fit))

#         non_zero_coefs <- coef_matrix[coef_matrix != 0, , drop = FALSE]
#         print("Coefficients when fitting: ")
#         print(non_zero_coefs)

#         data.frame(
#                         cvm = cv.fit$cvm[cv.fit$lambda == cv.fit$lambda.1se],
#                         lambda = cv.fit$lambda.1se,
#                         alpha = ai
#                         )
#         } 
#     cv.opt <- search[search$cvm == min(search$cvm),] 

#         # fit model by optimized alpha and lambda
#     set.seed(42)
#     yfit <- glmnet(
#     trainX,
#     trainY,
#     lambda = cv.opt$lambda,
#     alpha = cv.opt$alpha)

#     coef_matrix <- as.matrix(coef(yfit))

#     non_zero_coefs <- coef_matrix[coef_matrix != 0, , drop = FALSE]
#     print("Coefficients when optimal: ")
#     print(non_zero_coefs)

#     idf <- coef(yfit)
#     idx <- which(idf != 0)
#     selectf <- data.frame(
#             features = idf@Dimnames[[1]][idx], 
#             coefs = idf [idx]
#     )
# }

MWAS <- function(gwas, weight, geno){
        z <- gwas %*% weight
        z.cor <- cor(geno)
        se <- sqrt(weight %*%  z.cor %*%  weight)
        z <- z/se
        p=pnorm(abs(z),lower.tail=F)*2
        return(c(z, p))
}

In [5]:
df <- df[which(df$Chr == 7), ]

In [6]:
i <- 2

In [7]:
library("glmnet")
library("e1071")
library("doParallel")

set.seed(2018)
wind <- c(5000,10000)
# output directory
#outd <- "/dcl02/lieber/shan/shizhong/finemapping/GWAS/tags/scz3/mwas/chr22/1/"
outd <- "20-OUT_original_mwas_sanity_test/"

Loading required package: Matrix

Loaded glmnet 4.1-8

Loading required package: foreach

Loading required package: iterators

Loading required package: parallel



In [8]:
suppressWarnings(library(bsseq))

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:Matrix’:

    expand, unname


The following objects are masked from ‘package:data.table’:

    first, second


The following object is masked from ‘package:utils’:

    findMatches


The

In [9]:
# load data for mwas
# load("./rda/caudate_mwas_data_chr22.rda")
load(df$methylation_data[i])

p <- getMeth(BSobj2)


rownames(p) <- start(BSobj2)

sites_to_test <- which(start(BSobj2) >= (1987910 - 500) & start(BSobj2) <= (1987910 + 500))
p <- p[sites_to_test, ]

# candidate cg
cg <- as.numeric(rownames(p))

# regress out covariates
#load("covs_for_meqtl.rda")

In [10]:
sites_to_test

[1] 58792 58793 58794 58795 58796 58797 58798 58799 58800 58801 58802 58803
[13] 58804 58805 58806 58807 58808 58809 58810 58811 58812 58813 58814 58815
[25] 58816 58817 58818 58819 58820 58821 58822 58823 58824 58825 58826 58827
[37] 58828 58829 58830 58831 58832 58833 58834 58835 58836 58837 58838 58839
[49] 58840

In [11]:
covs <- fread(df$cov_file[i])
covs <- t(covs)
colnames(covs) <- covs[1, ]
covs <- covs[2:nrow(covs), ]
# transpose so we have same orientation as original code

In [12]:
BSobj2$brnum <- gsub("Br0", "Br", BSobj2$brnum)
colnames(covs) <- gsub("Br0", "Br", colnames(covs))

In [13]:
length(match(BSobj2$brnum,colnames(covs)))

[1] 297

In [14]:
mat <- match(BSobj2$brnum,colnames(covs)) 
covs <- t(covs[,mat])
p.residual=matrix(NA,dim(p)[1],dim(p)[2])

In [15]:
rownames(covs)[is.na(covs[, 'genoPC1'])] <- BSobj2$brnum[is.na(covs[, 'genoPC1'])]

In [16]:
colnames(p.residual) <- BSobj2$brnum

In [17]:
covs <- as.data.frame(covs)
# Convert all columns except Dx and Sex from character to numeric
cols_to_convert <- setdiff(names(covs), c("Dx", "Sex"))

for (col in cols_to_convert) {
  covs[[col]] <- as.numeric(covs[[col]])
}

# Print the modified data frame to check the conversion
#print(dat)


In [18]:
head(covs)

,genoPC1,genoPC2,genoPC3,methPC1,methPC2,methPC3,methPC4,methPC5,methPC6,methPC7,methPC8,methPC9,methPC10,Dx,Age,Sex
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
Br1122,0.0532758,0.00686226,0.088819600,-369.74476,-87.17607,168.759340,-269.504298,258.02473,-1.69242,48.97785,159.56483,-54.36247,35.931528,Control,48.61,M
Br2285,-0.0621117,0.06237480,-0.004629460,159.18604,-48.36568,261.556008,64.817606,108.47666,-52.11707,79.76524,91.72461,67.94607,26.261142,Control,53.29,M
Br1764,-0.0613176,0.03891250,-0.000738591,-74.62200,-107.85435,200.001413,58.971643,55.62332,-70.38619,185.05197,61.13412,31.48804,22.733084,Control,56.05,M
Br1464,0.0548900,-0.01760230,-0.104721000,-87.98447,262.20475,-115.927348,128.059707,239.34545,-97.21134,-82.06519,-24.13204,-139.33348,-15.655063,SCZ,55.69,F
Br5062,-0.0621787,0.05752900,-0.004001160,259.25733,-171.22695,8.888978,91.790650,109.38406,-96.18880,-67.54609,23.00965,92.64693,-5.282966,Control,82.66,M
Br1446,-0.0599802,-0.02904590,0.024323100,496.41183,-194.51406,9.438974,5.141084,60.36936,-42.95399,157.21084,-44.44427,47.59827,18.932008,Control,34.44,M


In [19]:
for (i in 1:dim(p)[1]) { # For each methylation site
    dat <- as.data.frame(cbind(y = p[i,], covs))
    
    # Check for rows with NAs (the ones for which we don't have covariate data)
    valid_rows <- complete.cases(dat)
    
    if (sum(valid_rows) > 0) {
        dat_valid <- dat[valid_rows,]
        model.res <- lm(y ~ ., data = dat_valid)
        
        # Store residuals in the corresponding positions
        p.residual[i, valid_rows] <- resid(model.res)
    }
}


# for(i in 1:dim(p)[1]){ # foro each methylation site
#         dat <- as.data.frame(cbind(p[i,],covs))
#         colnames(dat) <- c("y",paste0("x",1:ncol(covs)))
#         model.res <- lm(reformulate(paste0("x",1:ncol(covs)), "y"),dat)
#         p.residual[i,] = resid(model.res) 
# }

In [20]:
load("p1.rda")

In [21]:
wind <- 10000

In [22]:
# built predition models
idx.ea <- BSobj2$race == "CAUC"

In [23]:
snp2_sorted <- snp2[, order(names(snp2))]

In [24]:
colnames(snp2) <- gsub("Br0", "Br", colnames(snp2))

In [25]:
snp2 <- snp2[, colnames(snp2) %in% colnames(p.residual)]

In [26]:
dim(snp2)

[1]  56 297

In [27]:
snp2_positions <- stringr::str_split_fixed(rownames(snp2), ":", 3)[, 2]

## Test for one

In [28]:
dim(snp2)

[1]  56 297

In [29]:
rownames(snp2)

[1] "chr7:1963098:T:C" "chr7:1963408:C:T" "chr7:1964786:C:T" "chr7:1966112:T:C"
 [5] "chr7:1973362:G:A" "chr7:1975412:T:G" "chr7:1976335:C:T" "chr7:1977810:C:T"
 [9] "chr7:1978402:C:T" "chr7:1978783:G:T" "chr7:1979188:C:A" "chr7:1979235:G:A"
[13] "chr7:1980240:G:A" "chr7:1981269:G:C" "chr7:1981360:G:A" "chr7:1982428:C:G"
[17] "chr7:1985366:G:C" "chr7:1985405:A:G" "chr7:1987676:G:T" "chr7:1987689:G:A"
[21] "chr7:1987719:C:T" "chr7:1987813:T:C" "chr7:1989028:A:C" "chr7:1989333:G:T"
[25] "chr7:1989994:G:C" "chr7:1990232:T:C" "chr7:1990305:T:C" "chr7:1990372:G:C"
[29] "chr7:1990469:G:A" "chr7:1990652:G:T" "chr7:1993168:C:T" "chr7:1993230:A:T"
[33] "chr7:1994145:G:C" "chr7:1994382:C:T" "chr7:1994558:A:G" "chr7:1994714:G:A"
[37] "chr7:1994927:A:G" "chr7:1997034:T:G" "chr7:1997580:G:A" "chr7:1998126:G:A"
[41] "chr7:1999959:A:C" "chr7:2001797:A:G" "chr7:2002003:G:A" "chr7:2002813:C:T"
[45] "chr7:2002912:C:G" "chr7:2004461:G:C" "chr7:2005716:T:C" "chr7:2006553:G:C"
[49] "chr7:2006693:A:G" "chr7:2007195:C:T" "chr7:2008210:A:G" "chr7:2008240:G:A"
[53] "chr7:2008501:T:C" "chr7:2008585:G:C" "chr7:2008700:A:G" "chr7:2009071:A:G"

In [30]:
k <- 1

i <- 1

range1 <- ifelse(cg[i] - wind[k] > 0,cg[i] - wind[k],0)
range2 <- cg[i] + wind[k]

range1
range2

[1] 1977413

[1] 1997413

In [31]:
cat(i,"\n")
range1 <- ifelse(cg[i] - wind[k] > 0,cg[i] - wind[k],0)
range2 <- cg[i] + wind[k]
range1
range2

idx <- map2$POS > range1 & map2$POS < range2

length(idx)

1 


[1] 1977413

[1] 1997413

[1] 56

In [32]:
range1

[1] 1977413

In [33]:
range2

[1] 1997413

In [34]:
# go to next cg if no snps within window
if(sum(idx) <= 1){
        next
}
geno <- snp2[idx,]
rownames(geno) <- map2$POS[idx]
trainX <- t(geno)
trainY <- p.residual[i,]

In [35]:
dim(snp2)

dim(geno)

head(trainY[order(names(trainY))])

[1]  56 297

[1]  31 297

Br1003       Br1004       Br1007       Br1016       Br1017       Br1021 
 0.032737590  0.019424071  0.017150662  0.004945029 -0.007041670 -0.023944463

In [36]:
inspect <- t(snp2)
inspect <- inspect[order(rownames(inspect)), ]

In [37]:
fit <- elastic.net(trainX,trainY)

Warning message:
“executing %dopar% sequentially: no parallel backend registered”


In [38]:
fit

features,coefs
<chr>,<dbl>
(Intercept),-0.0019839249
1979188,0.0019631282
1980240,0.0001047671


We're getting different elasticnet results here than with our package. Why?

Let's go inside elastic.net function

In [39]:
dim(trainX)

[1] 297  31

Deeper still

In [40]:
trainX2 <- readRDS("20-IN_sanity_test_Xmatrix.rds")

In [41]:
trainy2 <- readRDS("20-IN_sanity_test_y.rds")

In [42]:
fold_id <- readRDS("20-IN_sanity_test_fold_id.rds")

In [43]:
set.seed(42)
cv.fit <- cv.glmnet(
                trainX,
                trainY,
    nfold = 5,
                #foldid = fold_id,                    
        type.measure = "mse",
                    paralle = TRUE,
                    alpha = 0.5
                    )

In [44]:
cv.fit


Call:  cv.glmnet(x = trainX, y = trainY, type.measure = "mse", nfolds = 5,      parallel = TRUE, alpha = 0.5) 

Measure: Mean-Squared Error 

      Lambda Index   Measure        SE Nonzero
min 0.004499     5 0.0004443 3.844e-05       1
1se 0.006528     1 0.0004468 3.781e-05       0

In [45]:
coef(cv.fit)

32 x 1 sparse Matrix of class "dgCMatrix"
                      s1
(Intercept) 3.388132e-20
1977810     .           
1978402     .           
1978783     .           
1979188     .           
1979235     .           
1980240     .           
1981269     .           
1981360     .           
1982428     .           
1985366     .           
1985405     .           
1987676     .           
1987689     .           
1987719     .           
1987813     .           
1989028     .           
1989333     .           
1989994     .           
1990232     .           
1990305     .           
1990372     .           
1990469     .           
1990652     .           
1993168     .           
1993230     .           
1994145     .           
1994382     .           
1994558     .           
1994714     .           
1994927     .           
1997034     .           

In [46]:
head(fold_id)

[1] 4 3 4 3 1 2

In [47]:
a <- 0.5
search <- foreach(ai = a, .combine = rbind) %dopar% {
    set.seed(42)
    cv.fit <- cv.glmnet(
                trainX2,
                trainy2,
                foldid = fold_id,                    
        type.measure = "mse",
                    paralle = TRUE,
                    alpha = ai
                    )
    data.frame(
                    cvm = cv.fit$cvm[cv.fit$lambda == cv.fit$lambda.1se],
                    lambda = cv.fit$lambda.1se,
                    alpha = ai
                    )
    } 

In [48]:
cv.opt <- search[search$cvm == min(search$cvm),] 

In [49]:
cv.opt

,cvm,lambda,alpha
,<dbl>,<dbl>,<dbl>
1,0.0004224127,0.007649007,0.5


In [50]:
cv.fit


Call:  cv.glmnet(x = trainX, y = trainY, type.measure = "mse", nfolds = 5,      parallel = TRUE, alpha = 0.5) 

Measure: Mean-Squared Error 

      Lambda Index   Measure        SE Nonzero
min 0.004499     5 0.0004443 3.844e-05       1
1se 0.006528     1 0.0004468 3.781e-05       0

In [51]:
coef(cv.fit) # This matches what we get out of tune_alpha, without fitting optimized parameters on whole data

32 x 1 sparse Matrix of class "dgCMatrix"
                      s1
(Intercept) 3.388132e-20
1977810     .           
1978402     .           
1978783     .           
1979188     .           
1979235     .           
1980240     .           
1981269     .           
1981360     .           
1982428     .           
1985366     .           
1985405     .           
1987676     .           
1987689     .           
1987719     .           
1987813     .           
1989028     .           
1989333     .           
1989994     .           
1990232     .           
1990305     .           
1990372     .           
1990469     .           
1990652     .           
1993168     .           
1993230     .           
1994145     .           
1994382     .           
1994558     .           
1994714     .           
1994927     .           
1997034     .           

In [52]:
a <- 0.5

In [53]:
search <- foreach(ai = a, .combine = rbind) %dopar% {
    set.seed(42)
    cv.fit <- cv.glmnet(
                    trainX2,
                    trainy2,
                    foldid = fold_id,
                    type.measure = "mse",
                    paralle = TRUE,
                    alpha = ai
                    )
    data.frame(
                    cvm = cv.fit$cvm[cv.fit$lambda == cv.fit$lambda.1se],
                    lambda = cv.fit$lambda.1se,
                    alpha = ai
                    )
    } 
cv.opt <- search[search$cvm == min(search$cvm),] 

    # fit model by optimized alpha and lambda
    set.seed(42)
    yfit = glmnet(
    trainX2,
    trainy2,
    lambda = cv.opt$lambda,
    alpha = cv.opt$alpha
            )       
    idf <- coef(yfit)
    idx <- which(idf != 0)
    selectf <- data.frame(
            features = idf@Dimnames[[1]][idx], 
            coefs = idf [idx]
    )

In [54]:
cv.opt # GOOD! should be 
#cvm	lambda	alpha
#<dbl>	<dbl>	<dbl>
#1	0.0004224127	0.007649007	0.5


,cvm,lambda,alpha
,<dbl>,<dbl>,<dbl>
1,0.0004224127,0.007649007,0.5


In [55]:
search

cvm,lambda,alpha
<dbl>,<dbl>,<dbl>
0.0004224127,0.007649007,0.5


Make sure we have same results not only from fitting but for optimal model with fixed alpha, lambda also

In [56]:
selectf

features,coefs
<chr>,<dbl>
(Intercept),-4.062561e-05
chr7:1977810:C:T,7.759077e-04
chr7:1981360:G:A,-1.070675e-03
chr7:1987719:C:T,-1.224151e-03
chr7:1987813:T:C,1.768736e-04
chr7:1990232:T:C,-1.298076e-03
chr7:1994382:C:T,1.105434e-03


In [57]:
dim(trainX)

[1] 297  31

In [58]:
models.all <- c()

In [59]:
rbind(models.all, cv.fit)

,lambda,cvm,cvsd,cvup,cvlo,nzero,call,name,glmnet.fit,lambda.min,lambda.1se,index
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<language>,<chr>,<elnet>,<dbl>,<dbl>,"<int[,1]>"
cv.fit,0.006527....,0.000446....,3.781117....,0.000484....,0.000409....,"0, 1, 1,....",cv.glmne....,Mean-Squ....,c(s0 = 3....,0.004499....,0.006527....,"5, 1"


Is 1se vs min for lambda the problem?

## Run for all

In [60]:
dim(snp2)

[1]  56 297

In [61]:
dim(snp2)

[1]  56 297

In [62]:
snp3 <- readRDS("20-IN_check_SNP_window_pos1987413_win50000.rds")

In [63]:
dim(snp2)

[1]  56 297

In [64]:
snp3 <- t(snp3)

In [65]:
map3 <- data.frame(POS = stringr::str_split_fixed(rownames(snp3), ":", 3)[, 2])

In [66]:
elastic.net(trainX, trainY)

In [67]:
trainX[1:10, 1:10]

,1977810,1978402,1978783,1979188,1979235,1980240,1981269,1981360,1982428,1985366
Br836,0,0.008,0.000,0.000,2,2.000,2,2.000,2.000,2
Br845,1,2.000,2.000,0.000,2,2.000,2,2.000,2.000,2
Br848,1,1.991,1.990,0.010,2,1.905,2,1.006,1.006,2
Br863,0,0.000,0.000,0.000,2,2.000,2,2.000,2.000,2
Br914,0,1.981,1.979,0.047,2,2.000,2,2.000,2.000,2
Br948,0,0.024,0.024,1.001,2,2.000,2,2.000,2.000,2
Br949,2,2.000,2.000,1.997,2,2.000,2,0.005,2.000,2
Br963,0,0.015,0.019,1.997,2,2.000,2,2.000,2.000,2
Br983,1,2.000,2.000,0.001,1,1.000,1,1.000,1.000,1
Br991,0,0.000,0.000,1.000,2,2.000,2,2.000,2.000,2


In [68]:
trainY[1:10]

Br1122        Br2285        Br1764        Br1464        Br5062 
-0.0138154121 -0.0225516889 -0.0004894733  0.0340564405 -0.0329548084 
       Br1446        Br1503        Br1946        Br5323        Br1297 
 0.0244990255 -0.0075554592  0.0427314728  0.0199602498 -0.0177260869

In [69]:
p.residual <- p.residual[, order(colnames(p.residual))]

In [70]:
for(k in 1:length(wind)){
    models.ea <- c()
    models.all <- c()
    for(i in 1:length(cg)){
    #for(i in 1){
            cat(i,"\n")
            print(paste0("This cg is: ", cg[i]))
            range1 <- ifelse(cg[i] - wind[k] > 0,cg[i] - wind[k],0)
            range2 <- cg[i] + wind[k]
            idx <- map3$POS > range1 & map3$POS < range2
            # go to next cg if no snps within window
            if(sum(idx) <= 1){
                    next
            }
            geno <- snp3[idx,] # changed snp2 to snp3
            rownames(geno) <- map3$POS[idx]
            trainX <- t(geno)
            trainY <- p.residual[i,]
            fit <- tryCatch(
                    elastic.net(trainX,trainY),
                    error = function(e) {return ("err")})
            if(!is.data.frame(fit)){
                if(fit == "err"){
                    next
                }
            }
            if(nrow(fit) == 0) next

            fit$cg <- cg[i]
            models.all <- rbind(models.all,fit)
            # EA only
            trainX <- trainX[idx.ea,]
            if(sum(apply(trainX,2,var)!=0) <= 1){
                    next
            }
            trainY <- trainY[idx.ea]
            fit <- tryCatch(
                    elastic.net(trainX,trainY),
                    error = function(e) {return ("err")})
            if(!is.data.frame(fit)){
                if(fit == "err"){
                    next
                }
            }
            if(nrow(fit) == 0) next
            fit$cg <- cg[i]
            models.ea <- rbind(models.ea,fit)
    }
}

1 
[1] "This cg is: 1987413"
2 
[1] "This cg is: 1987418"
3 
[1] "This cg is: 1987434"
4 
[1] "This cg is: 1987449"
5 
[1] "This cg is: 1987458"
6 
[1] "This cg is: 1987468"
7 
[1] "This cg is: 1987492"
8 
[1] "This cg is: 1987511"
9 
[1] "This cg is: 1987527"
10 
[1] "This cg is: 1987565"
11 
[1] "This cg is: 1987604"
12 
[1] "This cg is: 1987616"
13 
[1] "This cg is: 1987627"
14 
[1] "This cg is: 1987648"
15 
[1] "This cg is: 1987656"
16 
[1] "This cg is: 1987659"
17 
[1] "This cg is: 1987665"
18 
[1] "This cg is: 1987669"
19 
[1] "This cg is: 1987680"
20 
[1] "This cg is: 1987684"
21 
[1] "This cg is: 1987688"
22 
[1] "This cg is: 1987697"
23 
[1] "This cg is: 1987705"
24 
[1] "This cg is: 1987725"
25 
[1] "This cg is: 1987732"
26 
[1] "This cg is: 1987778"
27 
[1] "This cg is: 1987797"
28 
[1] "This cg is: 1987808"
29 
[1] "This cg is: 1987853"
30 
[1] "This cg is: 1987858"
31 
[1] "This cg is: 1987861"
32 
[1] "This cg is: 1987896"
33 
[1] "This cg is: 1987910"
34 
[1] "This cg is

In [71]:
wind[k]

[1] 10000

In [72]:
models.ea <- models.ea[models.ea[,1] != "(Intercept)",]
models.all <- models.all[models.all[,1] != "(Intercept)",]

In [73]:
if(!dir.exists(outd)) dir.create(outd)

In [74]:
head(models.all)

,features,coefs,cg
,<chr>,<dbl>,<dbl>
2,1977810,1.143246e-03,1987413
3,1979926,-2.984210e-03,1987413
4,1981360,-1.092704e-03,1987413
5,1987676,-1.484905e-03,1987413
6,1987719,-3.453372e-03,1987413
7,1990232,-9.106033e-05,1987413


In [75]:
dim(models.all)

[1] 644   3

In [76]:
# mwas by models of all samples
cg2 <- unique(models.all$cg)
mwas.all <- matrix(0,nrow=length(cg2),ncol=2)
for(i in 1:length(cg2)){
        pos <- models.all[models.all$cg == cg2[i],1]
        gwas <- snp.gwas2$z[is.element(snp.gwas2$pos_hg38, pos)]
        weight <- models.all[models.all$cg == cg2[i],2]
        geno <- snp.1kg.eur2[match(pos,map.1kg.eur2$POS),]
        mwas.all[i,] <- MWAS(gwas, weight, t(geno))
}
rownames(mwas.all) <- cg2
colnames(mwas.all) <- c("z","p")

# mwas by models of EA samples
cg2 <- unique(models.ea$cg)
mwas.ea <- matrix(0,nrow=length(cg2),ncol=2)
for(i in 1:length(cg2)){
        pos <- models.ea[models.ea$cg == cg2[i],1]
        gwas <- snp.gwas2$z[is.element(snp.gwas2$pos_hg38, pos)]
        weight <- models.ea[models.ea$cg == cg2[i],2]
        geno <- snp.1kg.eur2[match(pos,map.1kg.eur2$POS),]
        mwas.ea[i,] <- MWAS(gwas, weight, t(geno))
}
rownames(mwas.ea) <- cg2
colnames(mwas.ea) <- c("z","p")

# output models and mwas results
outf <- paste0(outd,"/models-cov1.all.wind.",wind[k])
write.csv(models.all,outf)
outf <- paste0(outd,"/models-cov1.ea.wind.",wind[k])
write.csv(models.ea,outf)
outf <- paste0(outd,"/mwas-cov1.all.wind.",wind[k])
write.csv(mwas.all,outf)
outf <- paste0(outd,"/mwas-cov1.ea.wind.",wind[k])
write.csv(mwas.ea,outf)

ERROR: Error in gwas %*% weight: non-conformable arguments


In [ ]:
dim(gwas)

In [ ]:
length(gwas)

In [ ]:
gwas

In [ ]:
length(gwas)

In [ ]:
length(weight)

In [ ]:
weight

In [ ]:
models.all

In [ ]:
mwas.all

In [ ]:
models.all

In [ ]:
mwas.all

In [ ]:
head(models.ea)